# Final Project - Random Forests
## Hudson Arney & Ian Golvach

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
df = pd.read_csv('Social_Network_Ads.csv')
df.head()

In [ ]:
df.head()

In [ ]:
sns.scatterplot(data=df, x='Age', y='EstimatedSalary', hue='Purchased')
plt.title('Age vs Estimated Salary')

In [ ]:
df.describe()

In [ ]:
df.info()

## Scratch implemntation of RF
Learning sourced from: https://en.wikipedia.org/wiki/Random_forest

Based on the wikipedia article, we will need to modify a standard decision tree model so that
 - Along with the gini coef. split, we select a random subset of the feature to consider when making splits
 - Several decision trees are trained with a 'bagging' approach, training the trees on a random swample w/ replacement of the data.
 
Following this, it makes the most sense that we first implement a method that will handle the sampling and training, as well as the predicting via plurality, and another method that will server as the model itself for the underlying decision trees (that way we can test the bagging concept with other models)

In [ ]:
import numpy as np
import scipy.stats

class bagging_trees:
    """Bootstrap aggregates a decision tree, bagging B times"""
    def __init__(self, model, B=5, is_classifier=True):
        self.B = B
        self.model = model
        self.fitted_models = []
        self.is_classifier = is_classifier

    def fit(self, X, y):
        # Create B models fitted on n sets created by randomly sampling with replacement from X and y
        if(X.shape[0] != y.shape[0]):
            raise Exception("X and y do not have the same number of columns")
        for i in range(self.B):
            self.__progress(i)
            bag_X = []
            bag_y = []
            for i in range(X.shape[0]):
                sample_ind = np.random.randint(X.shape[0])
                bag_X.append(X[sample_ind])
                bag_y.append(y[sample_ind])
            new_model = self.model()
            new_model.fit(np.array(bag_X), np.array(bag_y))
            self.fitted_models.append(new_model)
        self.__progress(self.B+1)
            
    def predict(self, X_pred):
        # Return the average of predictions if regression, otherwise return plurality of predictions
        # It may be advantageous to later replace these with np functions
        predictions = []
        for i in range(self.B):
            predictions.append(self.fitted_models[i].predict(X_pred))
        if(self.is_classifier):
            return np.ravel(scipy.stats.mode(np.array(predictions).T,1,keepdims=False)[0])
        else:
            return np.mean(np.array(predictions).T,1)
        
    def __progress(self, current):
        if(current-1 == self.B):
            print("[=] Sub-model "+str(self.B)+'/'+str(self.B)+' fitted. Fitting complete!')
        else:
            spinner = ' '
            # |/-\
            spinny = current%4
            if(spinny==0):
                spinner = '|'
            elif(spinny==1):
                spinner = '/'
            elif(spinny==2):
                spinner = '-'
            elif(spinny==3):
                spinner = '\\'
            print("["+spinner+"] Sub-model "+str(current+1)+'/'+str(self.B)+' fitting...', end='\r')

        
        

This code is just a proof of concept that the class actually works, which is why train/test splitting was not performed.

In [ ]:
from sklearn.tree import DecisionTreeClassifier

bag_trees = bagging_trees(DecisionTreeClassifier)

In [ ]:
X = np.array(df[['Age','EstimatedSalary']])
y= np.array(df['Purchased'])

In [ ]:
bag_trees.fit(X, y)

In [ ]:
bag_trees.predict(X)

Now we implement a decision tree that picks features at random.

In [ ]:
y/2

In [ ]:
np.unique(y, return_counts=True)

In [ ]:
test_array = [1,3,4,2,5]
test_array.sort()
test_array.remove(2)
test_array

In [ ]:
import math

class rdt_node:
    #type(X)==np.ndarray
    """
    Node for random_decision_tree, has a boundary, variable number, and between 0 and 2 children
    Children of this node can either be another node, or a value.
       
    var_num: the variable that is compared at this node
    boundary: the boundary along the variable, <= is left, otherwise right
    left: the left child of the node, or the value of the leaf
    right: the right child of the node, or the value of the leaf
    """
    def __init__(self, var_num=0, boundary=0, left=None, right=None):
        self.var_num = var_num
        self.boundary = boundary
        self.left = left
        self.right = right


class random_decision_tree:
    """
    Decision Tree that follows random forest standards
    
    depth_limit: how deep the forest is allowed to go before stopping
    feature_subset_func: the function used to determine how many feature to use based on number of features
    """
    def __init__(self, depth_limit=5, feature_subset_func=lambda x: math.floor(pow(x,1/2))):
        self.depth_limit = 5
        self.root = rdt_node();
        self.num_features = -1;
        self.feature_subset_func = feature_subset_func
    
    def fit(self, X_train, y_train):
        #X_train is a 2D numpy ndarray of shape (observations, features)
        #y_train is a 1D numpy array of the true value for the feature
        self.num_features = X_train.shape[1]
        self.__fit(X_train, y_train, 0, self.root)
        
        
    def __fit(self, X_train, y_train, depth, node):
        #for subsequent _fits, pass in the subset that is on that side of the boudary
        #depth increments each time fit is called
        #if the depth==depth_limit, do not call _fit again, but assign the plurality label and/or average value to child
        
        # first, decide which random subset of variables you will use for calculating the boundary
        feature_candidates = self.__array_for_range(X_train.shape[1])
        features = []
        
        for i in range(self.feature_subset_func(X_train.shape[1])):
            index = np.random.randint(len(feature_candidates))
            features.append(feature_candidates[index])
            feature_candidates.remove(feature_candidates[index])
        
        
        # then, find the desirable boundary for using gini coef (sub-method)
        gini_result = self.__find_gini(X_train, y_train, features)
        # if at depth, assign leafs, otherwise call fit on the subsets of the new boundary
        node.var_num = gini_result[1]
        node.boundary = gini_result[0]
        if(self.depth_limit == depth):
            #assign leaves based on plurity feature / average feature (class/regress)
            #classification assumed currently
            node.left = scipy.stats.mode(y_train[X_train[:,node.var_num]<=node.boundary],keepdims=False)[0]
            node.right = scipy.stats.mode(y_train[X_train[:,node.var_num]>node.boundary],keepdims=False)[0]
        else:
            # if leafs pure, assign their unique value, else call fit
            if(gini_result[2] and gini_result[3]):
                # both are leaves
                node.left = scipy.stats.mode(y_train[X_train[:,node.var_num]<=node.boundary],keepdims=False)[0]
                node.right = scipy.stats.mode(y_train[X_train[:,node.var_num]>node.boundary],keepdims=False)[0]
            elif(gini_result[2]):
                # only left is leaf, call fit on right
                node.left = scipy.stats.mode(y_train[X_train[:,node.var_num]<=node.boundary],keepdims=False)[0]
                node.right = rdt_node()
                self.__fit(X_train[X_train[:,node.var_num]>node.boundary],y_train[X_train[:,node.var_num]>node.boundary], depth + 1, node.right)
            elif(gini_result[3]):
                # only right is leaf, call fit on left
                node.right = scipy.stats.mode(y_train[X_train[:,node.var_num]>node.boundary],keepdims=False)[0]
                node.left = rdt_node()
                self.__fit(X_train[X_train[:,node.var_num]<=node.boundary],y_train[X_train[:,node.var_num]<=node.boundary], depth + 1, node.left)
            else:
                # no leaves, call fit on both
                node.left = rdt_node()
                node.right = rdt_node()
                self.__fit(X_train[X_train[:,node.var_num]<=node.boundary],y_train[X_train[:,node.var_num]<=node.boundary], depth + 1, node.left)
                self.__fit(X_train[X_train[:,node.var_num]>node.boundary],y_train[X_train[:,node.var_num]>node.boundary], depth + 1, node.right)
        # REMEMBER, _fit is called with the subset of the training X and y data that is on the respective side of the boundary
        
    def __array_for_range(self, num_for_array):
        # Sometimes the simple solution works
        ret = []
        for i in range(num_for_array):
            ret.append(i)
        return ret
    
    def __find_gini(self, X_train, y_train, features):
        # features is a list of integers denoting which column numbers (feature) to consider
        # returns a tuple of (boundary, feature, left_leaf, right_leaf) of types (int,float,bool,bool)
        # left_leaf, right_leaf are returned true if everything on the side of a boundary has the same y value
        unincluded_features = self.__array_for_range(X_train.shape[1])
        for x in features:
            unincluded_features.remove(x)
        #candidates are of form (boundary,feature), they are the halfway point between all parts of a feature
        boundary_candidates = []
        #boundaries are of form (boundary,feature,gini_coef)
        boundaries = []
        
        #setting up candidates
        #sorted_values = None
        for i in features:
            possible_values = np.unique(X_train[:,i])
            sorted_values = np.sort(possible_values)
            #print(sorted_values)
            #print(sorted_values.shape)
            #print(sorted_values.shape)
            if(sorted_values.shape[0] == 2):
                boundary_candidates.append(((sorted_values[0]+sorted_values[1])/2,i))
            else:
                for j in range(sorted_values.shape[0]-1):
                    boundary_candidates.append(((sorted_values[j]+sorted_values[j+1])/2,i))
        while(len(boundary_candidates)==0):
            if(len(unincluded_features)==0):
                raise Exception("Multiple duplicate observations, boundary cannot be found")
            #print("NO CANDIDATES")
            #print(X_train)
            #print(y_train)
            #print(sorted_values)
            # Testing has indicated that this is a result of no variance on the chosen features
            # As the sklearn decision tree does, we must allow more features until we atleast one candidate
            i = unincluded_features[0]
            possible_values = np.unique(X_train[:,i])
            sorted_values = np.sort(possible_values)
            #print(sorted_values.shape)
            if(sorted_values.shape[0] == 2):
                boundary_candidates.append(((sorted_values[0]+sorted_values[1])/2,i))
            else:
                for j in range(sorted_values.shape[0]-1):
                    boundary_candidates.append(((sorted_values[i]+sorted_values[i+1])/2,i))
            unincluded_features.remove(i)
            
        #print(boundary_candidates)
        #Find gini for all candidates
        for candidate in boundary_candidates:
            boundaries.append((candidate[0],candidate[1],self.__gini_index_for_boundary(X_train, y_train, candidate[0], candidate[1])))
        
        #sort candidates using hidden method
        boundaries.sort(key=self.__sorter)
        
        #set ret and check to see if either sub-region is 'pure'
        best_boundary = boundaries[0]
        left_leaf = (len(np.unique(y_train[X_train[:,best_boundary[1]]<=best_boundary[0]])) == 1)
        right_leaf = (len(np.unique(y_train[X_train[:,best_boundary[1]]>best_boundary[0]])) == 1)
        
        return(best_boundary[0],best_boundary[1],left_leaf,right_leaf)
        
        
    def __gini_index_for_boundary(self, X_train, y_train, boundary, feature_num):
        #classification assumed
        
        #calculate proportions of feature_num <= boundary for 1, proportion of feature_num > boundary for 2
        # code pulled from https://stackoverflow.com/questions/67586928/how-can-you-find-what-percentage-of-a-numpy-array-has-some-value-x
        # uniques, counts = np.unique(array, return_counts=True)
        # percentages = dict(zip(uniques, counts * 100 / len(array)))
    
        # r1 is the subset of y masked by X[:,feature_num]<= or > the boundary, because we use it to find the proportions
        R_1 = y_train[X_train[:,feature_num]<=boundary]
        N_1 = R_1.shape[0]
        proportions_1 = np.unique(R_1,return_counts=True)[1]/N_1
        
        R_2 = y_train[X_train[:,feature_num]>boundary]
        N_2 = R_2.shape[0]
        proportions_2 = np.unique(R_2,return_counts=True)[1]/N_2
        
        N = X_train.shape[0]
        
        #compute ginis
        g_1 = 0
        for prop in proportions_1:
            g_1 = g_1 + prop*(1-prop)
            
        g_2 = 0
        for prop in proportions_2:
            g_2 = g_2 + prop*(1-prop)
            
        #return weighted average of ginis
        return ((N_1/N)*(g_1))+((N_2/N)*(g_2))
        
    def __sorter(self, the_tuple):
        return the_tuple[2]
        
    def predict(self, X_pred):
        return np.array([self.__predict(x) for x in X_pred])
    
    def __predict(self, x_pred):
        # For any vector of features given, transverses the tree in order to return the value
        # x_pred is a vector
        tolkein = self.root # could replace this with 'walker' or something else
        while(type(tolkein)==type(self.root)):
            if(x_pred[tolkein.var_num] <= tolkein.boundary):
                tolkein = tolkein.left
            else:
                tolkein = tolkein.right
        return tolkein
    

In [ ]:
from sklearn.metrics import accuracy_score
rdt = random_decision_tree()
rdt.fit(X,y)
predicted = rdt.predict(X)
print(accuracy_score(y,predicted))

In [ ]:
rdt.root.var_num

Put it all together

In [ ]:
random_forest = bagging_trees(random_decision_tree, 100)
random_forest.fit(X,y)
prediction = random_forest.predict(X)
print(accuracy_score(y,prediction))